# making DF

In [7]:
import glob
import galsim
import piff
import meds
import numpy as np
import astropy.io.fits as fits

import sys
sys.path.append('/global/homes/m/mgatti/BFD_pipeline/')
#import BFD_pipeline   
#from BFD_pipeline.image_processing_routines import CollectionOfImages,MedsStamp,GalaxyModelsTable,check_on_exposures,save_moments_targets,collapse,grid_search
from BFD_pipeline_old.read_meds_utils import Image, MOF_table, DetectionsTable
MOF_deep_field = MOF_table('/global/cfs/cdirs/des/BFD_Y6/deep_fields_coadd/mof/run-ugriz-mof02.fits')

    
    
deep_fields_coadd_path = '/global/cfs/cdirs/des//BFD_Y6/deep_fields_coadd/'

raw_images =  '/global/cfs/cdirs/des/mgatti/BFD_targets/DF/files/'
piff_models = '/global/cfs/cdirs/des/mgatti/BFD_targets/DF/output/'
bands = ['g','r','i','z']

ff = ['COSMOS_UltraVISTA_Willv2', 'SN-X3','SN-C3','SN-E2']
for f_ in ff:
    fields = [f_]

    dictionary_runs = dict()

    for field in fields:
        folders = glob.glob(deep_fields_coadd_path+field+'/*/')
        for folder in folders:
            tile = (folder.split(deep_fields_coadd_path+field+'/')[1]).split('/')[0]

            dictionary_runs[tile] = dict()
            for band in bands:
                dictionary_runs[tile][band] = dict()
                dictionary_runs[tile][band]['meds'] = glob.glob(folder+'/*{0}_meds*'.format(band))
                dictionary_runs[tile][band]['psf'] = glob.glob(folder+'/*{0}_psfcat*'.format(band))
                try:

                    dictionary_runs[tile][band]['piff'] = glob.glob( piff_models+'/{0}_{1}.piff*'.format(tile,band))
                    dictionary_runs[tile][band]['raw_images'] = glob.glob( raw_images+'/{0}*{1}.fits.fz'.format(tile,band))
                except:
                    pass




    import copy
    import os
    for tile in dictionary_runs.keys():
    #if 1==1:
        xx = '/global/cfs/cdirs/des/BFD_Y6/deep_fields_coadd_PIFF/{2}/{1}/{1}_{0}_meds-Y3A2_DEEP_PIFF.fits'.format('z',tile,fields[0])
        if not os.path.exists(xx):

            for band in ['g','r','i','z']:
                xx = '/global/cfs/cdirs/des/BFD_Y6/deep_fields_coadd_PIFF/{2}/{1}/{1}_{0}_meds-Y3A2_DEEP_PIFF.fits'.format(band,tile,fields[0])
                if not os.path.exists(xx):
                    if len((dictionary_runs[tile][band]['piff']))>0:
                        raw_images = galsim.fits.read(dictionary_runs[tile][band]['raw_images'][0])
                        psf_array = piff.read(dictionary_runs[tile][band]['piff'][0])
                        df = fits.open(dictionary_runs[tile][band]['meds'][0])
                        wcs_ref = raw_images.wcs




                        #
                        # save the  psf ---------
                        # let's make a piff stamp for each entry


                        tab_detections = DetectionsTable({})
                        m = meds.MEDS(dictionary_runs[tile][band]['meds'][0])
                        for index in range(m.size):
                            Wide_g = Image(index, meds = [m], bands = [band])
                            tab_detections.add_image(Wide_g)
                        tab_detections.add_MOF_models(MOF_deep_field)

                        psf_row_size = np.zeros((len(df[1].data['id']),2),dtype=int)
                        psf_col_size = np.zeros((len(df[1].data['id']),2),dtype=int)
                        psf_start_row = np.zeros((len(df[1].data['id']),2),dtype=int)

                        psf_cutout_row = np.zeros((len(df[1].data['id']),2))
                        psf_cutout_col = np.zeros((len(df[1].data['id']),2))
                        count = 0

                        import frogress
                        for i in frogress.bar(range(len(df[1].data['id']))):
                            if band =='z':
                                color_ = tab_detections.images[i].MAG_I-tab_detections.images[i].MAG_Z
                            else:
                                color_ = tab_detections.images[i].MAG_G-tab_detections.images[i].MAG_I

                            # make PiFF model:
                            y = df[1].data['orig_row'][i][0]+1.
                            x = df[1].data['orig_col'][i][0]+1.

                            if band == 'z':
                                img = psf_array.draw(x,y , center=None,iz=color_,stamp_size = 25)
                            else:
                                img = psf_array.draw(x,y , center=None,gi=color_,stamp_size = 25)

                            dx = img.true_center.x-img.bounds.xmin
                            dy = img.true_center.y-img.bounds.ymin
                            img = img.array
                            psf_cutout_row[i] = dy
                            psf_cutout_col[i] = dx
                            psf_row_size[i] = img.shape[1]
                            psf_col_size[i] = img.shape[0]
                            psf_start_row[i] = count


                            count += img.shape[1]*img.shape[0]

                            if i == 0:
                                psf_array_img = img.reshape(img.shape[1]*img.shape[0])
                            else:
                                psf_array_img = np.hstack([psf_array_img,img.reshape(img.shape[1]*img.shape[0])])





                        df = fits.open(dictionary_runs[tile][band]['meds'][0])


                        new_hdulist = fits.HDUList()

                        new_column = fits.Column(name='psf_row_size', format='2J', array=psf_row_size.astype(int))
                        new_column1 = fits.Column(name='psf_col_size', format='2J', array=psf_col_size.astype(int))
                        new_column2 = fits.Column(name='psf_start_row', format='2K', array=psf_start_row.astype(int))
                        new_column3 = fits.Column(name='psf_cutout_row', format='2D', array=psf_cutout_row.astype(int))
                        new_column4 = fits.Column(name='psf_cutout_col', format='2D', array=psf_cutout_col.astype(int))
                        new_table = fits.BinTableHDU.from_columns(df[1].data.columns+new_column+new_column1+new_column2+new_column3+new_column4)


                        # Add all HDUs from the original file to the new list, except the one you're replacing
                        for i, hdu in enumerate(df):
                            if i != 1:
                                new_hdulist.append(hdu)
                            else:
                                new_hdulist.append(new_table)


                        for i in range(len(df[1].data['id'])):
                            y = df[1].data['orig_row'][i][0]+1.
                            x = df[1].data['orig_col'][i][0]+1.

                            jac = wcs_ref.local( galsim.PositionD(x,y))
                            new_hdulist[1].data['dudrow'][i] = jac.dudy      
                            new_hdulist[1].data['dudcol'][i] = jac.dudx
                            new_hdulist[1].data['dvdrow'][i] = jac.dvdy
                            new_hdulist[1].data['dvdcol'][i] = jac.dvdx

                        extname = df[1].header['EXTNAME']
                        new_hdulist[1].header['EXTNAME'] = extname

                        # add PSF extension

                        new_hdu = fits.CompImageHDU(psf_array_img.astype(np.float32))
                        new_hdu.header['FZQVALUE'] = 1
                        new_hdu.header['FZALGOR'] = 'RICE_1'
                        new_hdu.header['FZQMETHD'] = 'SUBTRACTIVE_DITHER_2'
                        new_hdu.header['FZTILE'] = (10240, 1) 
                        new_hdu.header['EXTNAME'] = 'psf'
                        new_hdulist.append(new_hdu)
                        pp = '/global/cfs/cdirs/des/BFD_Y6/deep_fields_coadd_PIFF/{2}/{1}/'.format(band,tile,fields[0])
                        if not os.path.exists(pp):
                            os.mkdir(pp)
                        path_save = '/global/cfs/cdirs/des/BFD_Y6/deep_fields_coadd_PIFF/{2}/{1}/{1}_{0}_meds-Y3A2_DEEP_PIFF.fits'.format(band,tile,fields[0])
                        new_hdulist.writeto(path_save,overwrite=True)



In [8]:
0

0

# making fofs

In [30]:
# source activate des-y6-fitvd -------
import os
import glob
mode = 'SN-E2'
mode = 'SN-X3'
#mode = 'SN-C3'
#mode = 'COSMOS_UltraVISTA_Willv2'
f = glob.glob('/global/cfs/cdirs/des/BFD_Y6/deep_fields_coadd_PIFF/{0}/*'.format(mode))
len(f)

for i in range(len(f)):
    t = f[i].split('/')[-1]
    print ("fitvd-make-fofs --conf '/global/homes/m/mgatti/fresh_BFD/BFD_pipeline/data/config_sof.yaml'   --output '/pscratch/sd/m/mgatti/BFD/data/fof-list/{1}_shredx-fofslist.fits' --plot dummy.png '{0}/{1}_i_meds-Y3A2_DEEP_PIFF.fits'".format(f[i],t))


fitvd-make-fofs --conf '/global/homes/m/mgatti/fresh_BFD/BFD_pipeline/data/config_sof.yaml'   --output '/pscratch/sd/m/mgatti/BFD/data/fof-list/SN-X3_C54_r3688p01_shredx-fofslist.fits' --plot dummy.png '/global/cfs/cdirs/des/BFD_Y6/deep_fields_coadd_PIFF/SN-X3/SN-X3_C54_r3688p01/SN-X3_C54_r3688p01_i_meds-Y3A2_DEEP_PIFF.fits'
fitvd-make-fofs --conf '/global/homes/m/mgatti/fresh_BFD/BFD_pipeline/data/config_sof.yaml'   --output '/pscratch/sd/m/mgatti/BFD/data/fof-list/SN-X3_C17_r3688p01_shredx-fofslist.fits' --plot dummy.png '/global/cfs/cdirs/des/BFD_Y6/deep_fields_coadd_PIFF/SN-X3/SN-X3_C17_r3688p01/SN-X3_C17_r3688p01_i_meds-Y3A2_DEEP_PIFF.fits'
fitvd-make-fofs --conf '/global/homes/m/mgatti/fresh_BFD/BFD_pipeline/data/config_sof.yaml'   --output '/pscratch/sd/m/mgatti/BFD/data/fof-list/SN-X3_C39_r3688p01_shredx-fofslist.fits' --plot dummy.png '/global/cfs/cdirs/des/BFD_Y6/deep_fields_coadd_PIFF/SN-X3/SN-X3_C39_r3688p01/SN-X3_C39_r3688p01_i_meds-Y3A2_DEEP_PIFF.fits'
fitvd-make-fofs --c

# collate

In [281]:
import glob
import galsim
import piff
import meds
import numpy as np
import astropy.io.fits as fits

import sys
sys.path.append('/global/homes/m/mgatti/BFD_pipeline/')
#import BFD_pipeline   
#from BFD_pipeline.image_processing_routines import CollectionOfImages,MedsStamp,GalaxyModelsTable,check_on_exposures,save_moments_targets,collapse,grid_search
from BFD_pipeline_old.read_meds_utils import Image, MOF_table, DetectionsTable
MOF_deep_field = MOF_table('/global/cfs/cdirs/des/BFD_Y6/deep_fields_coadd/mof/run-ugriz-mof02.fits')

    
    
deep_fields_coadd_path = '/global/cfs/cdirs/des//BFD_Y6/deep_fields_coadd/'

raw_images =  '/global/cfs/cdirs/des/mgatti/BFD_targets/DF/files/'
piff_models = '/global/cfs/cdirs/des/mgatti/BFD_targets/DF/output/'
bands = ['g','r','i','z']

ff = ['SN-C3']# , 'SN-X3','SN-C3']
for f_ in ff:
    fields = [f_]

    dictionary_runs = dict()

    for field in fields:
        folders = glob.glob(deep_fields_coadd_path+field+'/*/')
        for folder in folders:
            tile = (folder.split(deep_fields_coadd_path+field+'/')[1]).split('/')[0]

            dictionary_runs[tile] = dict()
            for band in bands:
                dictionary_runs[tile][band] = dict()
                dictionary_runs[tile][band]['meds'] = glob.glob(folder+'/*{0}_meds*'.format(band))
                dictionary_runs[tile][band]['psf'] = glob.glob(folder+'/*{0}_psfcat*'.format(band))
                try:

                    dictionary_runs[tile][band]['piff'] = glob.glob( piff_models+'/{0}_{1}.piff*'.format(tile,band))
                    dictionary_runs[tile][band]['raw_images'] = glob.glob( raw_images+'/{0}*{1}.fits.fz'.format(tile,band))
                except:
                    pass


In [282]:
import subprocess
import os
import argparse
from astropy.io import fits
from astropy.table import QTable,Table
from astropy.table import vstack
import numpy as np

import glob
for tile in dictionary_runs.keys():
    f = glob.glob('/pscratch/sd/m/mgatti/BFD/data/fitdv_df/{0}*'.format(tile))

    d1 =  QTable()
    d2 =  QTable()
    d3 =  QTable()
    d4 =  QTable()

    for f_ in f:
        hdul =  fits.open(f_) 


        d1_ = QTable(hdul[1].data)
        d2_ = QTable(hdul[2].data)
        d3_ = QTable(hdul[3].data)
        d4_ = QTable(hdul[4].data)

        d1 = vstack([d1, d1_])
        d2 = vstack([d2, d2_])
        d3 = vstack([d3, d3_])
        d4 = vstack([d4, d4_])

        d1_ = None
        d2_ = None
        d3_ = None
        d4_ = None
        #print(len(d2))
    d1 = Table(d1 )
    d2 = Table(d2 )
    d3 = Table(d3 )
    d4 =  Table(d4 ) 

    hdu1 = fits.BinTableHDU(d1)
    hdu2 = fits.BinTableHDU(d2)
    hdu3 = fits.BinTableHDU(d3)
    hdu4 = fits.BinTableHDU(d4)

    hdulist = fits.HDUList([fits.PrimaryHDU(), hdu1, hdu2])


    hdulist.writeto('/pscratch/sd/m/mgatti/BFD/data/fitdv_df_collated_2/{0}_sof.fits'.format(tile), overwrite=True)
